Game defined as :

    *State: 
        - Grid size
        - Positions of living cells
    *Rules: eg
        - A cell survives if it has 2 or 3 neighbours -> dies if 1 or 0 neighbours 
        - Dies if it has more than 3 neighbours.
        - If a dead cell has 3 neighbours it becomes alive.
    *Stop test:
        - If current state = previous state 
        

In [1]:
%matplotlib inline
import IPython.display
from io import BytesIO as bio
import PIL.Image
import numpy as np
from abc import ABC, abstractmethod
from copy import copy
import matplotlib.pyplot as plt
import matplotlib
import time
# matplotlib.use('GTK3Agg')

In [2]:
class State(ABC):
    @abstractmethod
    def copy(self):
        pass

    @abstractmethod
    def apply_rules(self, rules, max_size):
        pass

    @abstractmethod
    def equals(self, other):
        pass

    @abstractmethod
    def get_neighbours(self, elem, max_size):
        pass


class DenseNumpyState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return DenseNumpyState(np.copy(self.grid))

    def equals(self, other):
        if other is None:
            return False
        return np.array_equal(self.grid, other.grid)

    def apply_rules(self, rules, max_size,):
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

    def get_neighbours(self, elem, max_size):
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l


class SparseSetState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return SparseSetState(copy(self.grid))

    def get_neighbours(self, elem, max_size):
        # Returns the neighbours of a live cell if they lie within the bounds of the grid specified by max_size
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l

    def equals(self, other):
        if other is None:
            return False
        return self.grid == other.grid

    def apply_rules(self, rules, max_size):
        # Calls the actual rules and provides them with the grid and the neighbour function
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

# %%pixie_debugger
class Rule(ABC):
    @abstractmethod
    def apply_rules(self, grid, max_size, get_neighbours):
        pass


class DenseNumpyRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #copied_state = state.copy()
        #grid = state.grid

        grid_ret = copy(grid)
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                nb = get_neighbours((i, j), max_size)
                counter = 0
                for n in nb:
                    if grid[n] == True:
                        counter += 1
                if (counter < 2 or counter > 3):
                    grid_ret[i][j] = False
                if (counter == 3):
                    grid_ret[i][j] = True
        return grid_ret


class SparseSetRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #grid = state.grid
        counter = {}
        for elem in grid:
            if elem not in counter:
                counter[elem] = 0
            nb = get_neighbours(elem, max_size)
            for n in nb:
                if n not in counter:
                    counter[n] = 1
                else:
                    counter[n] += 1
        for c in counter:
            if (counter[c] < 2 or counter[c] > 3):
                grid.discard(c)
            if counter[c] == 3:
                grid.add(c)
        return grid

class Game:
    def __init__(self, initial_state, rules,max_size):
        self.initial_state = initial_state
        self.rules = rules
        self.max_size = max_size
    def run_game(self, it):
        state = self.initial_state
        previous_state = None
        progression = []
        i = 0
        while (not state.equals(previous_state) and i < it):
            i += 1
            previous_state = state.copy()
            progression.append(previous_state.grid)
            state = state.apply_rules(self.rules,self.max_size)
        progression.append(state.grid)
        return progression

def random_grid(size):
    randoms = np.random.randint(2, size=size*size)
    board = set()
    for index, elem in enumerate(randoms):
        if elem == 1:
            x = index%size
            y = int(index/size)
            board.add((x, y))
    return board

In [3]:
#transform array to a gif and save to a file
def save_gif(array,file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    frames = []
    for frame in range(array.shape[0]):
        img = PIL.Image.fromarray(array[frame])
        img = img.resize((500, 500))
        frames.append(img)
    img.save(file_name, save_all=True, duration=33.33, append_images=frames, loop=0,size=(500,500))

In [4]:
#transform array to a gif and save to a file
def save_img(array, file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
#     print(array.shape[0])
    img = PIL.Image.fromarray(array[array.shape[0] - 1 ])
    img = img.resize((500, 500))
    img.save(file_name, size=(500,500))

In [5]:
MAX_ITER = 1000
MAX_SIZE = 20
GIF_FOLDER = "gif/"
IMG_FOLDER = "img/"

def generate(number):
    number = str(number)
    init = np.zeros((MAX_SIZE, MAX_SIZE), dtype=bool)
    board = random_grid(MAX_SIZE)

    rules2 = SparseSetRules()
    game = Game(SparseSetState(board), rules2,MAX_SIZE)
    t = time.time()
    rw = game.run_game(MAX_ITER)
    print("Symulation:\t{}\tIterations:\t{}\tTime:\t{}".format(number, len(rw), time.time()-t))

    #transform sparse representation to array for plotting
    res = np.zeros((len(rw), MAX_SIZE, MAX_SIZE), dtype=bool)
    for l in range(0,len(rw)):
        for key in rw[l]:
            res[l,key[0], key[1]] = True 

    save_img(res,IMG_FOLDER + number + ".png")
    save_gif(res,GIF_FOLDER + number + ".gif")

In [8]:
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(generate, range(0, 1000))
            

Symulation:	4	Iterations:	31	Time:	0.021683692932128906
Symulation:	2	Iterations:	239	Time:	0.16175580024719238
Symulation:	1	Iterations:	83	Time:	0.20867371559143066
Symulation:	3	Iterations:	1001	Time:	0.18020915985107422
Symulation:	6	Iterations:	192	Time:	0.15969038009643555
Symulation:	8	Iterations:	33	Time:	0.03596234321594238
Symulation:	10	Iterations:	83	Time:	0.09087085723876953
Symulation:	0	Iterations:	256	Time:	0.43305468559265137
Symulation:	11	Iterations:	1001	Time:	0.1780529022216797
Symulation:	5	Iterations:	1001	Time:	0.5492134094238281
Symulation:	7	Iterations:	1001	Time:	0.45683717727661133
Symulation:	9	Iterations:	1001	Time:	0.371593713760376
Symulation:	12	Iterations:	61	Time:	0.01299595832824707
Symulation:	13	Iterations:	228	Time:	0.05398869514465332
Symulation:	14	Iterations:	96	Time:	0.03442835807800293
Symulation:	15	Iterations:	230	Time:	0.044715166091918945
Symulation:	16	Iterations:	94	Time:	0.021297454833984375
Symulation:	17	Iterations:	1001	Time:	0.1440

Symulation:	145	Iterations:	1001	Time:	0.14627480506896973
Symulation:	146	Iterations:	1001	Time:	0.26792049407958984
Symulation:	147	Iterations:	38	Time:	0.02184009552001953
Symulation:	148	Iterations:	1001	Time:	0.13493633270263672
Symulation:	149	Iterations:	75	Time:	0.03308463096618652
Symulation:	150	Iterations:	117	Time:	0.05608034133911133
Symulation:	151	Iterations:	91	Time:	0.02547144889831543
Symulation:	152	Iterations:	1001	Time:	0.13828706741333008
Symulation:	153	Iterations:	1001	Time:	0.11435317993164062
Symulation:	154	Iterations:	1001	Time:	0.12726283073425293
Symulation:	155	Iterations:	1001	Time:	0.18180179595947266
Symulation:	156	Iterations:	44	Time:	0.01261591911315918
Symulation:	157	Iterations:	1001	Time:	0.11829161643981934
Symulation:	158	Iterations:	90	Time:	0.028493881225585938
Symulation:	159	Iterations:	1001	Time:	0.12652325630187988
Symulation:	160	Iterations:	1001	Time:	0.4170079231262207
Symulation:	161	Iterations:	82	Time:	0.03789091110229492
Symulation

Symulation:	286	Iterations:	1001	Time:	0.2244861125946045
Symulation:	287	Iterations:	364	Time:	0.09574055671691895
Symulation:	288	Iterations:	1001	Time:	0.25405406951904297
Symulation:	289	Iterations:	1001	Time:	0.2261502742767334
Symulation:	290	Iterations:	1001	Time:	0.11425924301147461
Symulation:	291	Iterations:	1001	Time:	0.17586874961853027
Symulation:	292	Iterations:	81	Time:	0.04829239845275879
Symulation:	293	Iterations:	213	Time:	0.06057333946228027
Symulation:	294	Iterations:	72	Time:	0.025682926177978516
Symulation:	295	Iterations:	144	Time:	0.09133052825927734
Symulation:	296	Iterations:	95	Time:	0.03503847122192383
Symulation:	297	Iterations:	116	Time:	0.046460628509521484
Symulation:	298	Iterations:	1001	Time:	0.13506197929382324
Symulation:	299	Iterations:	114	Time:	0.054305315017700195
Symulation:	300	Iterations:	1001	Time:	0.1398012638092041
Symulation:	301	Iterations:	1001	Time:	0.15244579315185547
Symulation:	302	Iterations:	61	Time:	0.020740747451782227
Symulatio

Symulation:	427	Iterations:	88	Time:	0.03491377830505371
Symulation:	428	Iterations:	118	Time:	0.044100284576416016
Symulation:	429	Iterations:	1001	Time:	0.08768153190612793
Symulation:	430	Iterations:	105	Time:	0.04129147529602051
Symulation:	431	Iterations:	88	Time:	0.04741024971008301
Symulation:	432	Iterations:	1001	Time:	0.15989351272583008
Symulation:	433	Iterations:	72	Time:	0.029685258865356445
Symulation:	434	Iterations:	1001	Time:	0.055699825286865234
Symulation:	435	Iterations:	1001	Time:	0.27227282524108887
Symulation:	436	Iterations:	64	Time:	0.02923583984375
Symulation:	437	Iterations:	176	Time:	0.07072997093200684
Symulation:	438	Iterations:	39	Time:	0.018997907638549805
Symulation:	439	Iterations:	41	Time:	0.01787877082824707
Symulation:	440	Iterations:	1001	Time:	0.21698498725891113
Symulation:	441	Iterations:	54	Time:	0.03062605857849121
Symulation:	442	Iterations:	1001	Time:	0.21532320976257324
Symulation:	443	Iterations:	105	Time:	0.041532039642333984
Symulation:	4

Symulation:	568	Iterations:	1001	Time:	0.165360689163208
Symulation:	569	Iterations:	1001	Time:	0.19185614585876465
Symulation:	570	Iterations:	1001	Time:	0.2799055576324463
Symulation:	571	Iterations:	1001	Time:	0.1662585735321045
Symulation:	572	Iterations:	1001	Time:	0.16028809547424316
Symulation:	573	Iterations:	1001	Time:	0.14573049545288086
Symulation:	574	Iterations:	1001	Time:	0.2120223045349121
Symulation:	575	Iterations:	1001	Time:	0.16571831703186035
Symulation:	576	Iterations:	55	Time:	0.03386950492858887
Symulation:	577	Iterations:	110	Time:	0.03413558006286621
Symulation:	578	Iterations:	1001	Time:	0.19040703773498535
Symulation:	579	Iterations:	1001	Time:	0.1119546890258789
Symulation:	580	Iterations:	169	Time:	0.09140777587890625
Symulation:	581	Iterations:	1001	Time:	0.14438724517822266
Symulation:	582	Iterations:	1001	Time:	0.13826823234558105
Symulation:	583	Iterations:	1001	Time:	0.1470170021057129
Symulation:	584	Iterations:	59	Time:	0.030603885650634766
Symulatio

Symulation:	710	Iterations:	139	Time:	0.07416462898254395
Symulation:	711	Iterations:	1001	Time:	0.14112424850463867
Symulation:	712	Iterations:	1001	Time:	0.24946951866149902
Symulation:	713	Iterations:	215	Time:	0.1251990795135498
Symulation:	714	Iterations:	1001	Time:	0.1105501651763916
Symulation:	715	Iterations:	1001	Time:	0.28128862380981445
Symulation:	716	Iterations:	298	Time:	0.2112746238708496
Symulation:	717	Iterations:	68	Time:	0.04633641242980957
Symulation:	718	Iterations:	98	Time:	0.050713539123535156
Symulation:	719	Iterations:	1001	Time:	0.12734246253967285
Symulation:	720	Iterations:	54	Time:	0.017264127731323242
Symulation:	721	Iterations:	1001	Time:	0.15743088722229004
Symulation:	722	Iterations:	79	Time:	0.04841876029968262
Symulation:	723	Iterations:	50	Time:	0.026059389114379883
Symulation:	724	Iterations:	1001	Time:	0.3011596202850342
Symulation:	725	Iterations:	51	Time:	0.029932022094726562
Symulation:	726	Iterations:	67	Time:	0.03940534591674805
Symulation:	72

Symulation:	852	Iterations:	1001	Time:	0.15125060081481934
Symulation:	853	Iterations:	1001	Time:	0.4172391891479492
Symulation:	854	Iterations:	1001	Time:	0.2948019504547119
Symulation:	855	Iterations:	1001	Time:	0.2525765895843506
Symulation:	857	Iterations:	126	Time:	0.09650278091430664
Symulation:	856	Iterations:	1001	Time:	0.39156150817871094
Symulation:	858	Iterations:	133	Time:	0.0546422004699707
Symulation:	859	Iterations:	1001	Time:	0.1972954273223877
Symulation:	860	Iterations:	61	Time:	0.032956600189208984
Symulation:	861	Iterations:	85	Time:	0.11601018905639648
Symulation:	862	Iterations:	117	Time:	0.07858705520629883
Symulation:	863	Iterations:	74	Time:	0.04987382888793945
Symulation:	864	Iterations:	1001	Time:	0.18884849548339844
Symulation:	865	Iterations:	1001	Time:	0.278001070022583
Symulation:	866	Iterations:	304	Time:	0.13608932495117188
Symulation:	867	Iterations:	1001	Time:	0.17397785186767578
Symulation:	868	Iterations:	1001	Time:	0.1583881378173828
Symulation:	86

Symulation:	994	Iterations:	1001	Time:	0.1743624210357666
Symulation:	996	Iterations:	1001	Time:	0.19287824630737305
Symulation:	995	Iterations:	1001	Time:	0.25826120376586914
Symulation:	997	Iterations:	64	Time:	0.047847747802734375
Symulation:	998	Iterations:	1001	Time:	0.1825118064880371
Symulation:	999	Iterations:	371	Time:	0.2552037239074707


In [7]:
# for i in range(1000):
#     generate(i)